In [ ]:
%pip install openai langchain google-api-python-client

In [2]:
# Agents - LLMs

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import AzureOpenAI

# The language model we're going to use to control the agent.
llm = AzureOpenAI(temperature=0, deployment_name="testdavinci003")

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Let's test it out!
agent.run(
    "What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?<|im_end|>"
)



> Entering new  chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: google_search
Action Input: "high temperature in SF yesterday"
Observation: Weather reports from the last weeks in San Francisco with highs and lows. High: 66.2ºf @1:40 PM Low: 55.04ºf @5:56 AM Approx. Precipitation / Rain Total: in. Time (PDT), Temp. (ºf), Humidity (%), Dew Point ( ... San Francisco Weather Forecasts. ... Almanac for June 19, 2023. Forecast. Average *. Record. Temperature. High. 65 °F. 72 °F. 96 °F. Low. 53 °F. 54 °F. Sep 6, 2022 ... Bay Area heat wave: More cities break all-time high temps Tuesday ... recorded an astounding 116 degrees, which it also recorded yesterday. Sep 6, 2022 ... "Early morning temperatures in the East Bay and higher terrain are running 5 to 10 degrees higher today than they were yesterday," Brooke ... Jun 11, 2019 ... San Francisco International Airport yesterday reached its highest temperature ever recorded in the m

'The temperature in SF yesterday raised to the .023 power is 1.1012343099196273.'

In [3]:
# Agents - Chat models

from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI

# First, let's load the language model we're going to use to control the agent.
chat = AzureChatOpenAI(temperature=1, deployment_name="gpt-35-turbo")

# Note: llm-math might randomly fail when using gpt model due to this known issue
# https://github.com/hwchase17/langchain/issues/3071
# llm = AzureOpenAI(temperature=0.1, deployment_name="gpt35")

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = AzureOpenAI(temperature=0, deployment_name="testdavinci003")
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Let's test it out!
agent.run(
    "What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?"
)



> Entering new  chain...
Thought: I need to find the high temperature in SF yesterday and then raise it to the power of 0.023.

Action:
```
{
  "action": "google_search",
  "action_input": "SF yesterday high temperature Fahrenheit"
}
```


Observation: Weather reports from the last weeks in San Francisco with highs and lows. ... Maximum temperature yesterday: 68 °F (at 11:56 am) San Francisco, San Francisco International Airport ... Weather, Sky Cond. Temperature (ºF), Relative Humidity, Wind Chill (°F), Heat San Francisco Weather Forecasts. ... access_time 8:05 PM PDT on June 19, 2023 (GMT -7) | Updated just now. --° | 51°. 58 °F ... TomorrowTue 06/20 High 67 °F. Be prepared with the most accurate 10-day forecast for San francisco, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. Temperature (°F), Actual, Historic Avg. Record. High Temp, 57, 71.9, 91. Low Temp, 55, 53.8, 45. Day Average Temp, 56, 62.9, - ... Sep 5, 2022 ... Even cool coastal San

'1.101914010425215'

In [8]:
# Memory - LLMs

import langchain

from langchain import ConversationChain
from langchain.llms import AzureOpenAI

# Enable debug
langchain.debug = True

llm = AzureOpenAI(
    temperature=1,
    model_name="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    model_kwargs={"stop": ["Human:", "AI"]},
)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("Hi there! I'm Phong")
conversation.run("Can you remind me what my name is?")
conversation.run("What is the weather in New York now?")

[chain/start] [1:chain:ConversationChain] Entering Chain run with input:
{
  "input": "Hi there! I'm Phong",
  "history": ""
}
[llm/start] [1:chain:ConversationChain > 2:llm:AzureOpenAI] Entering LLM run with input:
{
  "prompts": [
    "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n\nHuman: Hi there! I'm Phong\nAI:"
  ]
}
[llm/end] [1:chain:ConversationChain > 2:llm:AzureOpenAI] [1.11s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Greetings! I am an artificial intelligence designed for friendly conversation. It is nice to meet you, Phong. How may I assist you today? \n\n",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "compl

"  I apologize, but I don't have real-time access to weather data. However, I can give you some general weather information. New York weather-wise is on the cooler side due to its temperate climate. Rain and snow are common during the winter season. The average temperature of New York in winter is 1℃ (33.8℉) and in summer is 26℃ (78.8℉). \n\n\n\n\n\n"

In [9]:
# Debug buffer

print(conversation.memory.buffer)

Human: Hi there! I'm Phong
AI:  Greetings! I am an artificial intelligence designed for friendly conversation. It is nice to meet you, Phong. How may I assist you today? 


Human: Can you remind me what my name is?
AI:  Your name is Phong! Was I correct? 





Human: What is the weather in New York now?
AI:   I apologize, but I don't have real-time access to weather data. However, I can give you some general weather information. New York weather-wise is on the cooler side due to its temperate climate. Rain and snow are common during the winter season. The average temperature of New York in winter is 1℃ (33.8℉) and in summer is 26℃ (78.8℉). 








In [12]:
# Memory - Chat models

import langchain

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory

# Enable debug
langchain.debug = True

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "The following is a friendly conversation between a human and an AI. The AI is talkative and "
            "provides lots of specific details from its context. If the AI does not know the answer to a "
            "question, it truthfully says it does not know."
        ),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

chat = AzureChatOpenAI(
    temperature=1, model_name="gpt-35-turbo", deployment_name="gpt-35-turbo"
)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=chat)

conversation.run(input="Hi there! I'm Phong")
conversation.run(input="Can you remind me what my name is?")
conversation.run(input="What is the weather in New York now?")

[chain/start] [1:chain:ConversationChain] Entering Chain run with input:
{
  "input": "Hi there! I'm Phong",
  "history": []
}
[llm/start] [1:chain:ConversationChain > 2:llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\nHuman: Hi there! I'm Phong"
  ]
}
[llm/end] [1:chain:ConversationChain > 2:llm:AzureChatOpenAI] [758.6220000000001ms] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello Phong! It's good to meet you. How can I help you today?",
        "generation_info": null,
        "message": {
          "content": "Hello Phong! It's good to meet you. How can I help you today?",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "t

'I am not able to check the current weather on my own, as I do not have access to the internet. However, if you provide me with a few moments to search for the information, I can tell you what the current weather is like in New York City.'

In [13]:
# Debug buffer

print(conversation.memory.buffer)

[HumanMessage(content="Hi there! I'm Phong", additional_kwargs={}, example=False), AIMessage(content="Hello Phong! It's good to meet you. How can I help you today?", additional_kwargs={}, example=False), HumanMessage(content='Can you remind me what my name is?', additional_kwargs={}, example=False), AIMessage(content='Your name is Phong, just as you told me earlier.', additional_kwargs={}, example=False), HumanMessage(content='What is the weather in New York now?', additional_kwargs={}, example=False), AIMessage(content='I am not able to check the current weather on my own, as I do not have access to the internet. However, if you provide me with a few moments to search for the information, I can tell you what the current weather is like in New York City.', additional_kwargs={}, example=False)]
